In [1]:
from fuvtds_base_class import FUVTDSBase

In [2]:
monitor = FUVTDSBase(PIDs='fuvtds_analysis_list.dat')

  0%|          | 0/1913 [00:00<?, ?it/s]

+++ Scaling LP6 to LP4 using data from datasets: /grp/hst/cos2/cosmo/16830/ler156gwq_x1d.fits.gz /grp/hst/cos2/cosmo/16830/ler156giq_x1d.fits.gz
+++ Scaling LP6 to LP4 using data from datasets: /grp/hst/cos2/cosmo/16830/ler15aknq_x1d.fits.gz /grp/hst/cos2/cosmo/16830/ler15al8q_x1d.fits.gz
+++ Scaling LP6 to LP4 using data from datasets: /grp/hst/cos2/cosmo/16830/ler156gyq_x1d.fits.gz /grp/hst/cos2/cosmo/16830/ler156gmq_x1d.fits.gz
+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv21f8q_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv23mtq_x1d.fits.gz
+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv21f8q_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv23mtq_x1d.fits.gz
+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv21f8q_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv23mtq_x1d.fits.gz
+++ Scaling LP4 to LP3 using data from datasets: /grp/hst/cos2/cosmo/14854/ldcv21f8q_x1d.fits.gz /grp/hst/cos2/cosmo/14854/ldcv23m